In [1]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from dash import Dash, html, dcc, callback, Output, Input, State
import dash_bootstrap_components as dbc
import webbrowser

import pandas as pd
import numpy as np
import math
import orjson

import timeit
import random

In [2]:
#read track data
trackData = pd.read_csv('SmallRun7_RadarTrack.txt', sep = r'\s*,\s*', engine = 'python', low_memory = True, usecols = ['Id','TimeStamp', 'Range', 'BearingAngle', 'LateralRate', 'RangeRate', 'ModeUpdated', 'RangeAcceleration', 'Width'])

#read intensity data
intData = pd.read_csv('SmallRun7_Intensities.txt', sep = r'\s*,\s*', engine = 'python', low_memory = True, usecols = ['Timestamp', 'GroupId', 'TrackPower1', 'TrackPower2', 'TrackPower3', 'TrackPower4', 'TrackPower5', 'TrackPower6', 'TrackPower7'])

In [3]:
#initialize the dataframe that will hold the final graph data
df = pd.DataFrame()

#convert polar coords to cartesian 
#x and y values are swapped to rotate the graph 90 degrees counterclockwise (into a vertical orientation)
x = list(np.sin(np.deg2rad(trackData.BearingAngle)) * trackData.Range)
y = list(np.cos(np.deg2rad(trackData.BearingAngle)) * trackData.Range)

df['xVals'] = x
df['yVals'] = y

#combine the track data and final graph data frame
df = pd.concat([df, trackData], axis = 1)

#mod lat and range rates
vectorX = list(df.RangeRate * np.cos(np.deg2rad(df.BearingAngle)) + df.LateralRate)
vectorY = list(df.RangeRate * np.sin(np.deg2rad(df.BearingAngle)))

vectorVel = np.hypot(vectorX, vectorY) / 83

df['xVel'] = vectorX
df['yVel'] = vectorY

#parse intensity data
#sort df to match the intensity values
df = df.sort_values(['Id', 'TimeStamp'])

intensityData = []
columns = ['TrackPower1', 'TrackPower2', 'TrackPower3', 'TrackPower4', 'TrackPower5', 'TrackPower6', 'TrackPower7']

#iterate through the intensity data and add it to df
for i in range(10):
    temp = intData[intData.GroupId == i]
    
    if i == 9:
        intensityData = intensityData + temp['TrackPower1'].to_list()
    else:
        for col in columns:
            intensityData = intensityData + temp[col].to_list()

df['Intensity'] = intensityData

#determine the needed animation steps
animationFrames = list(df.TimeStamp.unique())

for i in range(len(animationFrames) - 1):
    ref = animationFrames[i + 1] - animationFrames[i]
    
    if (ref < 0.020) and (ref != 0):
        animationFrames[i + 1] = animationFrames[i]

animationFrames = np.unique(animationFrames).tolist()

#scale animationFrames and add it to df
df['Frames'] = animationFrames * 64

df = df.sort_index()

#purge unnecessary data
df = df[df.Range > 0]

#create an object zero to ensure all frames are animated
filler = [0] * len(animationFrames)

objectZero = {
    'Id' : filler,
    'TimeStamp' : animationFrames,
    'Range' : filler,
    'RangeRate' : filler,
    'BearingAngle' : filler,
    'LateralRate' : filler,
    'xVals' : filler,
    'yVals' : filler,
    'Intensity' : filler,
    'Frames' : animationFrames,
    'xVel' : filler,
    'yVel' : filler, 
    'RangeAcceleration' : filler,
    'ModeUpdated' : filler,
    'Width' : filler
}

objZero = pd.DataFrame(objectZero)

#merge object zero and the parsed data
df = pd.concat([df, objZero], axis = 0) 

#sort df for graphing
df = df.sort_values(['TimeStamp', 'Id'])
df = df.reset_index(drop = True)

In [4]:
def createGraph(xLen, yLen, graphColor, animationSpeed):
    #create a figure object that will contain the animation
    fig = go.Figure()

    #create and name the graph traces (store them in data)
    data = []
    
    for i in range(4):
            
        modePos = go.Scatter(x = [0], y = [0], mode = 'markers', marker = dict(cmax = 40, cmin = -10, showscale = False, colorscale = 'Bluered'), showlegend = False, name = f'M{i}Pos')
        modeVel = ff.create_quiver(x = [1], y = [1], u = [1], v = [1], showlegend = False, name = f'M{i}Vel', hoverinfo = 'skip')
        modeLatR = ff.create_quiver(x = [1], y = [1], u = [1], v = [1], showlegend = False, name = f'M{i}LatR', hoverinfo = 'skip')
        modeRanR = ff.create_quiver(x = [1], y = [1], u = [1], v = [1], showlegend = False, name = f'M{i}RanR', hoverinfo = 'skip')
        modeWidth = ff.create_quiver(x = [1], y = [1], u = [1], v = [1], showlegend = False, name = f'M{i}W', hoverinfo = 'skip')
        modeRanA = ff.create_quiver(x = [1], y = [1], u = [1], v = [1], showlegend = False, name = f'M{i}RanA', hoverinfo = 'skip')
        
        data.extend([modePos, modeVel.data[0], modeLatR.data[0], modeRanR.data[0], modeWidth.data[0], modeRanA.data[0]])
    
    data.append(go.Scatter(x = [0], y = [0], showlegend = False, hoverinfo = 'skip', mode = 'markers', marker = dict(cmax = 40, cmin = -10, colorbar = dict(title = 'Intensity'), colorscale = 'Bluered'), name = 'filler')) 

    #create the animation's frames
    frames = []

    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 0, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    } #the slider dict is defined early for assignment in the following loop

    #current slowdown is caused by the number of frames iterating through the loop
    #could potentially be solved by declaring all needed dictionaries outside of python and then running the visualization
    for frame in animationFrames:
        #creates the animation frames
        animationData = []

        currentFrame = df[df.Frames == frame]

        for i in range(4):
            currentMode = currentFrame[currentFrame.ModeUpdated == i]

            if len(currentMode) == 0:
                positionData = go.Scatter(x = [0], y = [0], mode = 'markers', name = f'M{i}Pos')
                velocityData = ff.create_quiver(x = [1], y = [1], u = [0], v = [0], name = f'M{i}Vel')
                LatRate = ff.create_quiver(x = [1], y = [1], u = [0], v = [0], name = f'M{i}LatR')
                RanRate = ff.create_quiver(x = [1], y = [1], u = [0], v = [0], name = f'M{i}RanR')
                WidthData = ff.create_quiver(x = [1], y = [1], u = [0], v = [0], name = f'M{i}W')
                RanAcc = ff.create_quiver(x = [1], y = [1], u = [0], v = [0], name = f'M{i}RanA')

            else:
                positionData = go.Scatter(x = currentMode.xVals, y = currentMode.yVals, mode = 'markers', marker = dict(color = currentMode.Intensity), name = f'M{i}Pos',
                                          customdata = np.stack((currentMode.Id, currentMode.TimeStamp, currentMode.BearingAngle, currentMode.Range), axis=-1), 
                                          hovertemplate = '<b>Target</b>: %{customdata[0]}<br>' + '<b>Time</b>: %{customdata[1]}<br>' + '<b>Angle</b>: %{customdata[2]:,.0f}<br>' + '<b>Range</b>: %{customdata[3]}<br>' + '<extra></extra>')
                velocityData = ff.create_quiver(x = currentMode.xVals, y = currentMode.yVals, u = currentMode.xVel, v = currentMode.yVel, scale = 20, name = f'M{i}Vel', line = dict(color = 'black'))
                LatRate = ff.create_quiver(x = currentMode.xVals, y = currentMode.yVals, u = currentMode.LateralRate / 8, v = [0] * len(currentMode), scale = 20, name = f'M{i}LatR', line = dict(color = 'black'))
                RanRate = ff.create_quiver(x = currentMode.xVals, y = currentMode.yVals, u = currentMode.RangeRate / 82 * np.cos(np.deg2rad(currentMode.BearingAngle)), v = currentMode.RangeRate / 82 * np.sin(np.deg2rad(currentMode.BearingAngle)), scale = 20, name = f'M{i}RanR', line = dict(color = 'black'))
                WidthData = ff.create_quiver(x = list(currentMode.xVals - (currentMode.Width / 2)), y = currentMode.yVals, u = currentMode.Width, v= [0] * len(currentMode), scale = 1, angle = math.radians(0), name = f'M{i}W')
                RanAcc = ff.create_quiver(x = currentMode.xVals, y = currentMode.yVals, u = currentMode.RangeRate / 26 * np.cos(np.deg2rad(currentMode.BearingAngle)), v = currentMode.RangeRate / 26 * np.sin(np.deg2rad(currentMode.BearingAngle)), scale = 20, name = f'M{i}RanA', line = dict(color = 'black'))

            animationData.extend([positionData, velocityData.data[0], LatRate.data[0], RanRate.data[0], WidthData.data[0], RanAcc.data[0]])

        frames.append(go.Frame(data = animationData, traces = [x for x in range(24)], name = frame))

        #creates the animation slider functionality
        slider_step = {"args": [
            [frame],
            {"frame": {"duration": 0, "redraw": False},
             "mode": "immediate",
             "transition": {"duration": 0}}
        ],
            "label": frame,
            "method": "animate"}
        sliders_dict["steps"].append(slider_step)

    #finally assign the data and frames to the figure object
    fig["layout"]["sliders"] = [sliders_dict]
    fig.update(data = data, frames = frames)

    #create animation buttons (play/pause)
    fig["layout"]["updatemenus"] = [
        {
            "buttons": [
                {
                    "args": [None, {"frame": {"duration": 500, "redraw": False},
                                    "fromcurrent": True, "transition": {"duration": 300,
                                                                        "easing": "quadratic-in-out"}}],
                    "label": "Play",
                    "method": "animate"
                },
                {
                    "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                      "mode": "immediate",
                                      "transition": {"duration": 0}}],
                    "label": "Pause",
                    "method": "animate"
                }
            ],

            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]

    #command to control animation speed
    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = animationSpeed
    fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 0 #speed of frame transition (should remain 0)

    #create design elements to style the radar

    #create main radar cone
    right_corner = math.cos(math.radians(45)) * 175
    left_corner_x = math.cos(math.radians(45)) * -175

    fig.add_shape(type = "line", x0 = 0, y0 = 0, x1 = right_corner, y1 = right_corner, line_color = graphColor)
    fig.add_shape(type = "line", x0 = 0, y0 = 0, x1 = left_corner_x, y1 = right_corner, line_color = graphColor)

    #add range labels to graph
    for i in range(8):

        range_placement = np.cos(np.deg2rad(45)) * i * 25
        range_label = 25 * i
        fig.add_trace(go.Scatter(x = [-1 * range_placement], y = [range_placement], mode = "markers+text", text = range_label, textposition = "bottom left", showlegend = False, hoverinfo = 'skip', line_color = graphColor, name = 'filler'))

    #create radar arcs
    curveAngles = [45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135]

    for i in range(7):

        range_placement = (i + 1) * 25

        degree_x = range_placement * np.cos(np.deg2rad(curveAngles))
        degree_y = range_placement * np.sin(np.deg2rad(curveAngles))

        fig.add_trace(go.Scatter(x = degree_x, y = degree_y, mode = "lines", showlegend = False, hoverinfo = 'skip', line = dict(dash = 'dash'), line_color = graphColor, name = 'filler'))

    #add degree labels to graph
    fig.add_trace(go.Scatter(x = [0], y = [175], mode = "markers+text", text = 0, textposition = "top center", showlegend = False, hoverinfo = 'skip', line_color = graphColor, name = 'filler'))

    for i in range(9):
        degree = (i + 1) * 5
        xVal = degree_x[8 - i]
        yVal = degree_y[8 - i]

        fig.add_trace(go.Scatter(x = [xVal], y = [yVal], mode = "markers+text", text = degree, textposition = "top right", showlegend = False, hoverinfo = 'skip', line_color = graphColor, name = 'filler'))

    for i in range(9):
        degree = (i + 1) * -5
        xVal = degree_x[10 + i]
        yVal = degree_y[10 + i]

        fig.add_trace(go.Scatter(x = [xVal], y = [yVal], mode = "markers+text", text = degree, textposition = "top left", showlegend = False, hoverinfo = 'skip', line_color = graphColor, name = 'filler'))

    #add labels (right, front, left)
    fig.add_annotation(text = "Radar Front", xref= "x", yref = "y", x = 0, y = 187.5, showarrow = False)
    fig.add_annotation(text = "Radar Left", xref= "x", yref = "y", x = -125, y = 187.5, showarrow = False)
    fig.add_annotation(text = "Radar Right", xref= "x", yref = "y", x = 125, y = 187.5, showarrow = False)

    #general graph specifications
    fig.update_yaxes(visible = True, showticklabels = True, gridcolor = 'black', scaleanchor = None, scaleratio = None, dtick = 25, showgrid = True, autorange = True)
    fig.update_xaxes(visible = True, showticklabels = True, gridcolor = 'black', scaleanchor = 'y', scaleratio = 0.8, dtick = 25, showgrid = True, autorange = True)

    fig.update_layout(title_text = None,
                      paper_bgcolor = 'rgba(0,0,0,0)',
                      plot_bgcolor = 'rgba(0,0,0,0)',
                      #autosize = True,
                      width = None, 
                      height = 850, 
                      margin=dict(l=20, r=20, t=20, b=20),
                      xaxis_range = [-xLen, xLen], 
                      yaxis_range = [-1, yLen], 
                      legend = dict(
                        x = 0.01,
                        y = 0.01,
                        bgcolor = "White",
                        bordercolor = "Black",
                        borderwidth = 2
                      )
    )
    
    return fig

In [5]:
#modify the data with test conditions (generates fake data)

#fake intensity data
color = []

for i in range(len(df)):
    color.append(random.randrange(-10, 40))
    
df['Intensity'] = color

#fake mode values
fakeModes = []

for i in range(len(df)):
    fakeModes.append(random.randrange(0, 3))
    
df['ModeUpdated'] = fakeModes

fakeLatRate = []

for i in range(len(df)):
    fakeLatRate.append(random.randrange(-8, 8))
    
df['LateralRate'] = fakeLatRate

fakeRanRate = []

for i in range(len(df)):
    fakeRanRate.append(random.randrange(-82, 82))
    
df['RangeRate'] = fakeRanRate

fakeWidth = []
for i in range(len(df)):
    fakeWidth.append(random.randrange(0, 20))
    
df['Width'] = fakeWidth

fakeRanA = []
for i in range(len(df)):
    fakeRanA.append(random.randrange(0, 20))
    
df['Range Acceleration'] = fakeRanA

In [6]:
#user specifications

#set figure dimensions
xLen = 140 #displayed length in the x dimension in m (default is 140)
yLen = 190 #displayed length in the y dimension in m (default is 190)

#assign the radar asthetic color - options formatted as "#34e48c", "PaleTurquoise", etc.)
graphColor = "black" 

#animation speed control
animationSpeed = 50 #time in ms

fig = createGraph(xLen, yLen, graphColor, animationSpeed)

In [7]:
#create a loading screen figure
loadingFig = go.Figure()

loadingFig.add_annotation(text = 'Visualization Loading', showarrow = False, x = 5, y = 9, font = dict(size = 60) )

loadingFig.update_layout(
    title_text = None,
    paper_bgcolor = 'rgba(0,0,0,0)',
    plot_bgcolor = 'rgba(0,0,0,0)',
    #autosize = True,
    width = None, 
    height = 850, 
    margin=dict(l=20, r=20, t=20, b=20),
    xaxis_range = [0, 10], 
    yaxis_range = [0, 10], 
    
)

loadingFig.update_yaxes(visible = True, showticklabels = False, scaleanchor = None, scaleratio = None)
loadingFig.update_xaxes(visible = True, showticklabels = False, scaleanchor = 'y', scaleratio = 8)

#create table figure
tableData = pd.DataFrame()
tableData = pd.concat([tableData, df.Id, df.TimeStamp, df.Frames, df.BearingAngle, df.Range, df.RangeRate, df.LateralRate, df.RangeAcceleration, df.Width, df.Intensity, df.ModeUpdated], axis = 1)
tableData = tableData[tableData.Id > 0]

table = dbc.Table.from_dataframe(tableData, striped = True, bordered = True, hover = True)

#list of traces that need to be toggled
dropdown_dict = {
    'Mode 0' : 0,
    'Mode 1' : 1, 
    'Mode 2' : 2,
    'Mode 3' : 3,
    'Velocity' : 'Vel',
    'Width' : 'W',
    'Range Rate' : 'RanR',
    'Lateral Rate' : 'LatR',
    'Range Acceleration' : 'RanA'
}

display_data = []

app = Dash(__name__, external_stylesheets = [dbc.themes.LUX])

port = 8054
webbrowser.open_new("http://localhost:{}".format(port))

app.layout = dbc.Container([
    
    dbc.Row(
        dbc.Col(
            dbc.Alert([
                html.H1('Radar Visualization', className = 'text-center font-weight-bolder text-primary'),
                html.H6('ERDC', className = 'text-center text-primary')
            ], color = 'primary', className = 'mt-3 mb-3'), width = 12
        )
    ),
    
    dbc.Row([
        dbc.Col([
            
            dbc.Alert([
                html.Label(children = 'Position Values', className = 'text-left mb-2 text-black'),
                dbc.Checklist(options = ['Mode 0', 'Mode 1', 'Mode 2', 'Mode 3'], value = [], id = 'position-selection', className = 'text-black', inline = True),
                html.Br(),

                html.Label(children = 'Display Options', className = 'text-left mb-2 text-black'),
                dbc.Checklist(options = ['Velocity', 'Width', 'Lateral Rate', 'Range Rate', 'Range Acceleration'], value = [], id = 'display-selection', className = 'text-black', inline = True),
                html.Br(),

                html.Label(children = 'X-Axis Slider', className = 'text-left mb-2 text-black'),
                dcc.RangeSlider(-140, 140, 1, marks = None, value=[-140, 140], id = 'xaxis', tooltip={"placement": "bottom", "always_visible": True}),
                html.Br(),

                html.Label(children = 'Y-Axis Slider', className = 'text-left mb-2 text-black'),
                dcc.RangeSlider(0, 190, 1, marks = None, value=[0, 190], id = 'yaxis', tooltip={"placement": "bottom", "always_visible": True}),
                html.Br(),
                
                html.Label(children = 'Animation Speed (in ms)', className = 'text-left mb-2 text-black'),
                dbc.Input(id = 'speed-toggle', placeholder = 'Enter animation speed...', type = 'number'),
                html.Br(),
                
                dbc.Checklist(options = ['Open Table'], id = 'table-button', value = ['Open Table'], className = 'text-black'),
                html.Br(),
                html.Br()
            ], color = 'light', style = {'height' : '850px'})
            
        ], width = 4),
        
        dbc.Col([
            dcc.Graph(id = 'graph-content', figure = loadingFig),
        ], width = 8),
        
    ]),
    
    dbc.Row([
        
        dbc.Collapse([
            dcc.Dropdown(options = animationFrames, value = [], multi = True, id = 'table-frame-selector'),
            html.Div(
                dbc.Table(table, id = 'frame-table', color = 'light'), style = {'maxHeight' : '500px', 'overflow' : 'scroll'},
            ),
        ], id = 'table-collapse', is_open = False)
        
    ])
    
], fluid = True)

@callback(
    Output('graph-content', 'figure'), 
    Input('position-selection', 'value'),
    Input('display-selection', 'value'),
    Input('xaxis', 'value'),
    Input('yaxis', 'value'),
    Input('speed-toggle', 'value')
)
def update_graph(mode, displayOptions, xRange, yRange, aniSpeed):
    #controls mode points shown 
    update_mode(mode)
    
    #controls other options (vel, width, etc) based on modes active
    update_options(displayOptions)
    
    #allows control of the x and y axis
    fig.update_layout(
        xaxis_range = [xRange[0], xRange[1]], 
        yaxis_range = [yRange[0], yRange[1]], 
    )
    
    #allows control of animation speed
    if type(aniSpeed) is int:
        fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = aniSpeed
    
    return fig

#function controlling mode points shown
def update_mode(value):
    deactivate = ['Mode 0', 'Mode 1', 'Mode 2', 'Mode 3']
    
    for element in value:
        fig.update_traces(visible = True, selector = dict(name = f'M{dropdown_dict[element]}Pos'))
        deactivate.remove(element)
        
        if element not in display_data:
            display_data.append(element)
        
    for element in deactivate:
        fig.update_traces(visible = False, selector = dict(name = f'M{dropdown_dict[element]}Pos'))
        
        if element in display_data:
            display_data.remove(element)

#function controlling options (vel, width, etc.)
def update_options(value):
    deactivate = [
        'M0Vel', 'M1Vel', 'M2Vel', 'M3Vel',
        'M0LatR', 'M1LatR', 'M2LatR', 'M3LatR', 
        'M0RanR', 'M1RanR', 'M2RanR', 'M3RanR',
        'M0W', 'M1W', 'M2W', 'M3W',
        'M0RanA', 'M1RanA', 'M2RanA', 'M3RanA'
    ]
    
    for element in value:
        for mode in display_data:
            
            fig.update_traces(visible = True, selector = dict(name = f'M{dropdown_dict[mode]}{dropdown_dict[element]}'))
            
            if f'M{dropdown_dict[mode]}{dropdown_dict[element]}' in deactivate:
                deactivate.remove(f'M{dropdown_dict[mode]}{dropdown_dict[element]}')
    
    for element in deactivate:
        fig.update_traces(visible = False, selector = dict(name = element))   

@callback(
    Output('frame-table', 'children'),
    Input('table-frame-selector', 'value')
)
def update_table(selectedFrames):
    
    if len(selectedFrames) == 0:
        return table
    
    else:
        tempTableData = pd.DataFrame()

        for element in selectedFrames:
            tempTableData = pd.concat([tempTableData, tableData[tableData.Frames == element]], axis = 0)
            
        tempTable = dbc.Table.from_dataframe(tempTableData, striped = True, bordered = True, hover = True)

        return tempTable

@callback(
    Output('table-collapse', 'is_open'),
    Input('table-button', 'value'),
)
def toggle_table(tableState):
    if 'Open Table' in tableState:
        return True
    else:
        return False

if __name__ == '__main__':
    app.run_server(port = port,  host = 'localhost', use_reloader = False)